# Single Tarso (Pastas) model for one time series

This notebook creates a minimal Pastas model (Tarso-style) using a random timeseries
`output_data/only_csv_wiertsema/time_series.csv` as the observed head.

Steps:
1. Find the repository root using relative paths.
2. Load the CSV and pick the first numeric column as the observed series.
3. Try to load a precipitation stress from `input_stressors/prec_station_249.csv` (if present), otherwise create a small synthetic stress.
4. Build a tiny Pastas model with a Gamma response and fit it.

Run the cells below in order.

In [1]:
# Cell 2: setup imports and repo discovery
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pastas as ps


warnings.filterwarnings('ignore')

# find repo root
repo_root = Path.cwd()
for candidate in [repo_root] + list(repo_root.parents):
    if (candidate / 'pyproject.toml').exists() or (candidate / '.git').exists():
        repo_root = candidate
        break
print('Repo root:', repo_root)

wiertsema_dir = repo_root / 'output_data' / 'only_csv_wiertsema'
fugro_dir = repo_root / 'output_data' / 'only_csv_fugro'    
singles_dir = repo_root / 'output_data' / 'only_csv_singles'
stressor_dir = repo_root / 'input_stressors'
out_fig = repo_root / 'output_data' / 'figures'
out_fig.mkdir(parents=True, exist_ok=True)

Repo root: d:\Users\jvanruitenbeek\data_validation


In [2]:
import pandas as pd
from pathlib import Path

# Name of the directory containing single well data
name_model = "86349-1 HB011PB01 HB_BE0147+68_BUKR_GMW_PB1_F-247"

# # --- Load and clean the head data ---------------------------------------------
# df_head = pd.read_csv(
#     Path(wiertsema_dir) / (name_model + ".csv"),
#     index_col=0,
#     parse_dates=True,
#     encoding="utf-8-sig"
# )

#--- Load and clean the head data ---------------------------------------------
df_head = pd.read_csv(
    Path(wiertsema_dir) / (name_model + ".csv"),
    usecols=["DateTime","head"],
    parse_dates=["DateTime"],
    dayfirst=True,
    index_col="DateTime",
    encoding="utf-8-sig"
)

# Take first numeric column and rename to 'head'
head = df_head.select_dtypes("number").iloc[:, 0].rename("head")

# Crop to first and last valid (non-NaN) data points
df_head = head.loc[head.first_valid_index(): head.last_valid_index()]


df_prec = pd.read_csv(Path(stressor_dir) / "prec_station_249_2020.csv",
                   index_col=0, parse_dates=True)["Precipitation"]

df_evap = pd.read_csv(Path(stressor_dir) / "evap_station_249_2020.csv",
                   index_col=0, parse_dates=True)["ET"]

df_prec_long = pd.read_csv(Path(stressor_dir) / "precipitation_long_clean.csv",
                   index_col=0, parse_dates=True)["precipitation"]

df_evap_long = pd.read_csv(Path(stressor_dir) / "evaporation_long_clean.csv",
                   index_col=0, parse_dates=True)["evaporation"]

ValueError: Usecols do not match columns, columns expected but not found: ['DateTime', 'head']

In [ ]:
head.plot()

In [ ]:
head.hist()

In [ ]:
# start_slice_date = "2024-02-15"
# end_slice_date = "2024-12-03"

# # Slicing the data 
# df_slice = df_head.loc[start_slice_date:end_slice_date]

# Modifying df_head to only include data between start_slice_date and end_slice_date
#df_slice.info()
df_head.sort_index().resample("D").last()

#Converting values to NaN
df_head_clean = df_head.where(df_head >= -450)

# Converting to m and plotting
df_head_m = df_head_clean / 100
df_head_m.plot()

In [ ]:
df_prec.plot()
df_prec.info()

In [ ]:
df_evap.plot()
df_evap.info()

In [ ]:
import pandas as pd
import pastas as ps

# --- Prepare series from your current variables --------------------------------
# If df_head is a DataFrame, grab the first numeric column; else keep as Series.
head = df_head_m
prec = df_prec.rename("precip")
evap = df_evap.rename("evap")

In [ ]:
# Initialize and configure the first model
ml = ps.Model(head, name="Model")
ts1 = ps.RechargeModel(
    prec=prec,
    evap=evap,
    rfunc=ps.Exponential(),
    recharge=ps.rch.Linear(),
    name="recharge",
)
ml.add_stressmodel(ts1)
ml.add_transform(ps.ThresholdTransform())
# mlt.del_noisemodel()
ml.solve(report=True, warmup=3650)

In [ ]:
sm = ml.stressmodels["recharge"]
prec, evap = sm.stress[0].series_original, sm.stress[1].series_original

# delete all the stressmodels, the constant and the transform
ml.del_stressmodel("recharge")
ml.del_constant()
ml.del_transform()

# then add a TarsoModel
sm = ps.TarsoModel(prec, evap, dmin=head.min(), dmax=head.max(), name="Tarso")
ml.add_stressmodel(sm)

# --- Constrain time scales & evap factor --------------------------------------
ml.set_parameter("Tarso_a0", pmin=0,  pmax=200.0)   # days
ml.set_parameter("Tarso_a1", pmin=10.0, pmax=200.0)  # days



# and solve and plot the results again
ml.solve(report=True)
ml.plots.results(figsize=(10,4))

# Saving model to file
ml.to_file(name_model + "_tarso_model.pas") 

In [ ]:
ml.get_parameters()

In [ ]:
ml.get_response_tmax("Tarso", cutoff=0.95)

In [ ]:
# Functie om karakterestieken van de block response te krijgen
def get_peakblock_t95(param):
    """
    Computes peak block and characteristic response time (t95) for
    an exponential model with given parameters.

    Parameters:
        param (dict): Dictionary containing model parameters.

    Returns:
        tuple: (Piek_e, t95_e, Piek_0, t95_0) - peak block and t95 for both upper (_e) and lower regime (_0).
    """
    A1, a1, A0, a0, d0, d1 = (
        param["Tarso_A1"],
        param["Tarso_a1"],
        param["Tarso_A0"],
        param["Tarso_a0"],
        param["Tarso_d0"],
        param["Tarso_d1"],
    )

    # Compute equivalent parameters
    A_e = 1 / ((1 / A0) + (1 / A1))
    (A1 / (A0 + A1)) * d0 + (A0 / (A0 + A1)) * d1
    a_e = (a1 / A1) * A_e

    # Compute characteristic response times
    t95_e = ps.Exponential().get_tmax(p=(A_e, a_e), cutoff=0.95)
    t95_0 = ps.Exponential().get_tmax(p=(A0, a0), cutoff=0.95)

    # Compute peak blocks
    Piek_e = ps.Exponential(meanstress=1, cutoff=0.95).block((A_e, a_e))[0]
    Piek_0 = ps.Exponential(meanstress=1, cutoff=0.95).block((A0, a0))[0]

    return Piek_e, t95_e, Piek_0, t95_0

In [ ]:
# --- Map ml.get_parameters() -> your dict format
vals = ml.get_parameters()  # array([A0, a0, d0, A1, a1, d1, f])

param = {
    "Tarso_A0": float(vals[0]),
    "Tarso_a0": float(vals[1]),
    "Tarso_d0": float(vals[2]),
    "Tarso_A1": float(vals[3]),
    "Tarso_a1": float(vals[4]),
    "Tarso_d1": float(vals[5]),
    # not used by your function, but available:
    "Tarso_f":  float(vals[6]),
}

# --- Now call your function
Piek_e, t95_e, Piek_0, t95_0 = get_peakblock_t95(param)

print(f"Peak block (equivalent): {Piek_e:.6g}")
print(f"t95 (equivalent):        {t95_e:.2f} days")
print(f"Peak block (A0,a0):      {Piek_0:.6g}")
print(f"t95 (A0,a0):             {t95_0:.2f} days")

In [ ]:
# Overall time span (days)
span_days = (df_head.index.max() - df_head.index.min()) / pd.Timedelta(days=1)
print("Total span (days):", span_days)